In [ ]:
!nvidia-smi

Sun Mar 10 11:50:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
! git clone https://huggingface.co/openai-community/gpt2

Cloning into 'gpt2'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 87 (delta 0), reused 0 (delta 0), pack-reused 84
Unpacking objects: 100% (87/87), 1.65 MiB | 1.81 MiB/s, done.
Filtering content: 100% (11/11), 5.23 GiB | 56.94 MiB/s, done.


In [54]:
! ls gpt2/

64-8bits.tflite  flax_model.msgpack	 onnx		    tf_model.h5
64-fp16.tflite	 generation_config.json  pytorch_model.bin  tokenizer_config.json
64.tflite	 merges.txt		 README.md	    tokenizer.json
config.json	 model.safetensors	 rust_model.ot	    vocab.json


# **Загрузка модели**

In [1]:
import transformers
import torch

In [2]:
device = 'cuda'

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'gpt2',
    torch_dtype=torch.float64,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': "<|endoftext|>"})

1

In [5]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt").to(device)
outputs = model(**inputs)

In [9]:
output = model.generate(**inputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
[tokenizer.decode(x) for x in output.detach()]

["Hello, my dog is cute. I'm not sure if she's a puppy or not. I"]

# **SFT**

In [11]:
import pandas as pd
from torch import nn

In [12]:
sft_df = pd.read_parquet('argilla_df.parquet')

In [10]:
sft_df

,prompt,response-1,response-2,category,original_response,external_id
0,What is Depreciation,What is Depreciation – 10 Important Facts to K...,What is Depreciation on a Car?\nDepreciation i...,open_qa,Depreciation is the drop in value of an asset ...,518
1,What do you know about the city of Aberdeen in...,"Aberdeen, the city in which I've lived for the...","As an AI language model, I don't have personal...",open_qa,Aberdeen is a city located in the North East o...,351
2,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,information_extraction,Thunderstorm season in the United States and C...,10567
3,"When did Peloton IPO?\nOn September 26, 2019, ...",When did Peloton IPO?\nPeloton launched its in...,When did Peloton IPO?\nPeloton IPO'd on May 26...,closed_qa,Peloton became a public company via an initial...,12412
4,What is the best way to answer an interview qu...,The best way to answer an interview question m...,Some of the best ways to answer an interview q...,creative_writing,The first recommended step is to ask clarifyin...,2521
...,...,...,...,...,...,...
7396,How do i accept the change,How do i accept the change in my life\nAccepti...,I's a great opportunity to improve. The only t...,brainstorming,Embrace the change and see the difference,15010
7397,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,information_extraction,"Brazil, Botafogo-SP, Corinthians, Fiorentina",9970
7398,Without quoting directly from the text give me...,Without quoting directly from the text give me...,Without quoting directly from the text give me...,summarization,"Brendon Small is a stand-up comedian, Creator...",14205
7399,Is Killing is Sin ? Is it ture,Is Killing is Sin ? Is it ture?\nKilling can b...,Is Killing is Sin ? Is it ture?\nKilling is no...,brainstorming,Killing a human being should not be sin becaus...,11253


In [13]:
len(sft_df)/3

2467.0

In [14]:
sft_df = sft_df[:2467]

In [15]:
sft_df.iloc[0]['prompt']

'What is Depreciation'

In [16]:
sft_df.iloc[0]['original_response']

"Depreciation is the drop in value of an asset due to wear and tear, age and obsolescence (going out of date) as recorded in an organization's financial records."

In [17]:
tokenizer.add_special_tokens({'pad_token': "<|endoftext|>"})
tokenizer.encode_plus(sft_df.iloc[0]['prompt'], sft_df.iloc[0]['original_response'], truncation=True, padding="max_length", max_length=128, return_tensors="pt")

{'input_ids': tensor([[ 2061,   318,  2129, 33950, 12156, 33950,   318,   262,  4268,   287,
          1988,   286,   281, 11171,  2233,   284,  5806,   290, 11626,    11,
          2479,   290, 10201, 16850,   594,   357,  5146,   503,   286,  3128,
             8,   355,  6264,   287,   281,  4009,   338,  3176,  4406,    13,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 5

In [18]:
from torch.utils.data import Dataset
from typing import Dict, Sequence, List, Iterator, Tuple

IGNORE_INDEX: int = -100

def _tokenize_fn(text: str, max_length: int, tokenizer: transformers.PreTrainedTokenizer) -> List[int]:
    return tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        add_special_tokens=False,
    )['input_ids']

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""
    def __init__(self, dataset, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        self.tokenizer = tokenizer

        bos_token = tokenizer.bos_token if tokenizer.bos_token is not None else ''
        self.sources = [f"{bos_token}{dataset[0]}" for example in dataset]
        self.targets = [f"{dataset[1]}{tokenizer.eos_token}" for example in dataset]

    def __len__(self) -> int:
        return len(self.sources)

    def __getitem__(self, i: int) -> Dict[str, List[int]]:
        source_ids = _tokenize_fn(self.sources[i], self.tokenizer.model_max_length, self.tokenizer)
        target_ids = _tokenize_fn(self.targets[i], self.tokenizer.model_max_length - len(source_ids), self.tokenizer)
        input_ids = torch.tensor(source_ids + target_ids)
        labels = torch.tensor([IGNORE_INDEX] * len(source_ids) + target_ids)
        return dict(input_ids=input_ids, labels=labels)


In [19]:
train_dl = SupervisedDataset(
    dataset=sft_df[['prompt', 'original_response']].values,
    tokenizer=tokenizer,
)

In [20]:
for x in train_dl:
  break

In [21]:
x

{'input_ids': tensor([50256, 17816,  2061,   318,  2129, 33950,     6,   198,   366, 12156,
         33950,   318,   262,  4268,   287,  1988,   286,   281, 11171,  2233,
           284,  5806,   290, 11626,    11,  2479,   290, 10201, 16850,   594,
           357,  5146,   503,   286,  3128,     8,   355,  6264,   287,   281,
          4009,   338,  3176,  4406,   526,    60, 17816,  2061,   466,   345,
           760,   546,   262,  1748,   286, 44985,   287,  8838,  8348,   198,
           705,    32,   527, 39060,   318,   257,  1748,  5140,   287,   262,
          2258,  3687,   286,  8838,    13,  1081,   286, 12131,   340,   550,
           257,  3265,   286,   939,    11,   830,   661,   290,   318,   262,
          2368,  4387,  1748,   287,  8838,    11,  2157, 23475,   357, 40064,
            11,   830,     8,   290, 23995,   357, 39132,    11,   830,   737,
         44985,   468,   587,  6412,   284,   355,   366,   464,  7698,  2254,
             1,  2233,   284,   262,  1

In [27]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'gpt2',
    torch_dtype=torch.float64,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [29]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-6)
OUTPUT_SIZE = 256
N_ITERATIONS = 10
cur_iteration = 0
for x in train_dl:
    if cur_iteration == N_ITERATIONS:
          break

    model.train()
    #################
    input_tokens = x['input_ids'].to(device)
    labels = x['labels'].clone().to(device)
    out_logits = model(input_ids=input_tokens).logits
    labels[labels == tokenizer.pad_token_id] = -100
    loss_value = loss(out_logits, labels)
    #################
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()

    model.eval()
    cur_iteration += 1

Loss value: 8.820287172356947
Loss value: 8.755766937020562
Loss value: 8.715806088868316
Loss value: 8.554888604164757
Loss value: 8.767579415921402
Loss value: 8.73946592751169
Loss value: 8.60818522848961
Loss value: 8.23811645478674
Loss value: 8.310116901029604
Loss value: 8.546378518786343


# **Reward Model**

In [30]:
import pandas as pd

In [31]:
rm_df = pd.read_parquet('argilla_df.parquet')

In [32]:
rm_df = rm_df[2467:]

In [33]:
from torch.utils.data import Dataset
from typing import Dict, Sequence, List, Iterator, Tuple

IGNORE_INDEX: int = -100
bos_token_id = 50256

def _tokenize_fn(text: str, max_length: int, tokenizer: transformers.PreTrainedTokenizer) -> List[int]:
    return tokenizer(
        text,
        max_length=max_length,
        truncation=True,
        add_special_tokens=False,
    )['input_ids']


class RewardDataset(Dataset):
    """Dataset for supervised fine-tuning."""
    def __init__(self, dataset, tokenizer: transformers.PreTrainedTokenizer):
        super(RewardDataset, self).__init__()
        self.tokenizer = tokenizer

        bos_token = tokenizer.bos_token if tokenizer.bos_token is not None else ''
        self.chosen = [f"{bos_token}{dataset[0]}\n###answer: {dataset[1]}" for example in dataset]
        self.rejected = [f"{bos_token}{dataset[0]}\n###answer: {dataset[2]}" for example in dataset]

    def __len__(self) -> int:
        return len(self.chosen)

    def __getitem__(self, i: int) -> Dict[str, List[int]]:
        chosen_ids = _tokenize_fn(self.chosen[i], self.tokenizer.model_max_length, self.tokenizer)
        rejected_ids = _tokenize_fn(self.rejected[i], self.tokenizer.model_max_length, self.tokenizer)

        min_length = min(len(chosen_ids), len(rejected_ids))

        return dict(
            input_ids_chosen=torch.tensor(chosen_ids[:min_length] + [bos_token_id]),
            input_ids_rejected=torch.tensor(rejected_ids[:min_length] + [bos_token_id])
        )


In [34]:
train_rm = RewardDataset(
    dataset=rm_df[['prompt', 'original_response', 'response-2']].values,
    tokenizer=tokenizer,
)

In [35]:
for x in train_rm:
      break

In [36]:
x

{'input_ids_chosen': tensor([50256, 17816,  5376,  1936,  2678,   287,   262,  8342, 42811,     6,
           198,   705, 17940,    11,  4881,    11,  3794,    11, 17837,   290,
          7137,   389,  2678,  5140,   287,   262,  8342, 42811,  2637,   198,
           705,  5376,  1936,  2678,   287,   262,  8342, 42811,   326,   389,
          5140,  5000,  1626,   262, 32909, 16291,    13,    59,    77,   464,
          1936,  2678,   287,   262,  8342, 42811,   326,   389,  5140,  5000,
          1626,   262, 32909, 16291,   389, 15238,    11, 30155,    11, 17333,
            11,  3284,    11,   290,  3340,  2637,    60,   198, 21017, 41484,
            25, 37250,  9487,  1958,   262,  1708,   355,   852,  2035,   852,
           257,  2099,   286, 21027,   393,   257,  2099,   286,  6512,    25,
           220, 15554,  2256,    11, 31176,    11, 26241,    11,  2497,  1462,
           849,    11,  2042, 22504,    11, 22812,  2305,    11,   267,  2777,
          4364,    11, 24215,  1

In [37]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'gpt2',
    torch_dtype=torch.float64,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [38]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-6)
OUTPUT_SIZE = 256
DEVICE = 'cuda'
N_ITERATIONS = 10
cur_iteration = 0
for x in train_rm:
    if cur_iteration == N_ITERATIONS:
          break

    model.train()
    #################
    input_ids_chosen = x['input_ids_chosen'].to(device)
    input_ids_rejected = x['input_ids_rejected'].to(device)

    rewards_chosen = model(input_ids=input_ids_chosen).logits
    rewards_rejected = model(input_ids=input_ids_rejected).logits

    loss_value = -nn.functional.logsigmoid(rewards_chosen - rewards_rejected).mean()
    #################
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()

    model.eval()
    cur_iteration += 1

Loss value: 12.560367009876542
Loss value: 12.83289847798418
Loss value: 17.986245992511734
Loss value: 11.951677311417672
Loss value: 13.093826860505558
Loss value: 12.679522068155327
Loss value: 9.535560513681435
Loss value: 8.263899231624476
Loss value: 11.218700475602228
Loss value: 12.972257917681253
